In [1]:
import os;

from glob import glob;

from skimage import color, io;
from scipy.misc import imresize;

import tflearn;
from tflearn.data_utils import shuffle, to_categorical;
from tflearn.layers.core import input_data, dropout, fully_connected;
from tflearn.layers.conv import conv_2d, max_pool_2d;
from tflearn.layers.estimator import regression;
from tflearn.data_preprocessing import ImagePreprocessing;
from tflearn.data_augmentation import ImageAugmentation;
from tflearn.metrics import Accuracy;

from sklearn.cross_validation import train_test_split;

import numpy;

import pickle;

In [8]:
files_path = '/Users/ravishchawla/workspace/machinelearning/datasets/dogs_vs_cats/train/';

cat_files_path = os.path.join(files_path, 'cats/cat*.jpg');
dog_files_path = os.path.join(files_path, 'dogs/dog*.jpg');

cat_files = sorted(glob(cat_files_path));
dog_files = sorted(glob(dog_files_path));

num_files = len(cat_files) + len(dog_files);
print(num_files);

size_image = 128;

20000


In [9]:
try:
    (all_x, all_y) = pickle.load(open('all_x_y.dat', 'rb'));
except:
    all_x = numpy.zeros((num_files, size_image, size_image, 3), dtype='float64');
    all_y = numpy.zeros(num_files);
    count = 0;

    for file in cat_files:
        try:
            img = io.imread(file);
            new_img = imresize(img, (size_image, size_image, 3));
            all_x[count] = numpy.array(new_img);
            all_y[count] = 0;
            count = count + 1;
        except:
            continue

    for file in dog_files:
        try:
            img = io.imread(file);
            new_img = imresize(img, (size_image, size_image, 3));
            all_x[count] = numpy.array(new_img);
            all_y[count] = 1;
            count = count + 1;
        except:
            continue
    pickle.dump((all_x, all_y), open('all_x_y.dat', 'wb'));

In [10]:
X, X_test, Y, Y_test = train_test_split(all_x, all_y, test_size = 0.1, random_state = 42);

Y = to_categorical(Y, 2);
Y_test = to_categorical(Y_test, 2);

In [5]:
img_prep = ImagePreprocessing();
img_prep.add_featurewise_zero_center();
img_prep.add_featurewise_stdnorm();

img_aug = ImageAugmentation();
img_aug.add_random_flip_leftright();
img_aug.add_random_rotation(max_angle=25.);

In [6]:
network = input_data(shape=[None, 64, 64, 3],
data_preprocessing=img_prep,
data_augmentation=img_aug);

conv_1 = conv_2d(network, 32, 3, activation='relu', name='conv_1');

network = max_pool_2d(conv_1, 2);

conv_2 = conv_2d(network, 64, 3, activation='relu', name='conv_2');

conv_3 = conv_2d(conv_2, 64, 3, activation='relu', name='conv_3');

network = max_pool_2d(conv_3, 2);

network = fully_connected(network, 512, activation='relu');

network = dropout(network, 0.5);

network = fully_connected(network, 2, activation='softmax');

accuracy = Accuracy(name='Accuracy');
network = regression(network, optimizer='adam',
loss='categorical_crossentropy',
learning_rate=0.0005, metric=accuracy);

model = tflearn.DNN(network, checkpoint_path='model_cat_dog_6.tflearn', max_checkpoints=3,
tensorboard_verbose=3, tensorboard_dir='tmp.tflearn_logs');

In [7]:
model.fit(X_inputs=X, Y_targets=Y, validation_set=(X_test, Y_test), batch_size=500,
n_epoch=100, run_id='model_cat_dog_6', show_metric=True);

model.save('model_cat_dog_6_final.tflearn');

Training Step: 3600  | total loss: 0.17350
| Adam | epoch: 100 | loss: 0.17350 - Accuracy: 0.9589 | val_loss: 0.43950 - val_acc: 0.8470 -- iter: 18000/18000
Training Step: 3600  | total loss: 0.17350
| Adam | epoch: 100 | loss: 0.17350 - Accuracy: 0.9589 | val_loss: 0.43950 - val_acc: 0.8470 -- iter: 18000/18000
--
